# Training parameterized quantum circuits

## Vocab
+ Minimize <phi(theta)|H|phi(theta)> (this is the loss function)
+ Quantum Fisher Information
+ Simultaneous Perturbation Stochastic Approximation

In [1]:
from qiskit.circuit.library import RealAmplitudes
ansatz = RealAmplitudes(num_qubits=2, reps=1,
                        entanglement='linear')
# RealAmplitudes -> TwoLocal -> NLocal -> BlueprintCircuit -> QuantumCircuit
print(ansatz.draw())
ansatz = ansatz.decompose()
ansatz.draw()

     ┌──────────────────────────────────────┐
q_0: ┤0                                     ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3]) │
q_1: ┤1                                     ├
     └──────────────────────────────────────┘


┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├
     ├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├
     └──────────┘└───┘└──────────┘

## Calculate finite difference gradient
### Opflow (deprecated)

In [2]:
import numpy as np
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, Z, I

hamiltonian = Z ^ Z
expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(ansatz)
pauli_basis = PauliExpectation().convert(expectation)

quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'),
                                   # we'll set a seed for reproducibility
                                   shots = 8192, seed_simulator = 2718,
                                   seed_transpiler = 2718)
sampler = CircuitSampler(quantum_instance)

def evaluate_expectation(theta):
    value_dict = dict(zip(ansatz.parameters, theta))
    result = sampler.convert(pauli_basis, params=value_dict).eval()
    print(result)
    return np.real(result)

print(ansatz.num_parameters)
point = np.random.random(ansatz.num_parameters)
INDEX = 2

EPS = 0.2
# make identity vector with a 1 at index ``INDEX``, otherwise 0
e_i = np.identity(point.size)[:, INDEX]
print(e_i)

plus = point + EPS * e_i
minus = point - EPS * e_i

print(plus)
print(minus)

finite_difference = (
    evaluate_expectation(plus) - evaluate_expectation(minus)) / (2 * EPS)
print(finite_difference)

C:\Users\vutha\AppData\Local\Temp\ipykernel_33344\3630295614.py:4: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, Z, I
C:\Users\vutha\AppData\Local\Temp\ipykernel_33344\3630295614.py:7: DeprecationWarning: The class ``qiskit.opflow.state_fns.operator_state_fn.OperatorStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(ansatz)
C:\Users\vutha\AppData\Local\Temp\ipykernel_33344\3630295614.py:7: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be remo

4
[0. 0. 1. 0.]
[0.69904579 0.83363198 0.44595015 0.72839291]
[0.69904579 0.83363198 0.04595015 0.72839291]
(0.30175781250000006+0j)
(0.141357421875+0j)
0.4010009765625001


### Quantum_info and primitives

In [3]:
from qiskit.quantum_info import Pauli
from qiskit_aer.primitives import Estimator

hamiltonian = Pauli('ZZ')

estimator = Estimator()
def evaluate_expectation(theta):
    result = estimator.run(ansatz, hamiltonian, theta).result().values
    print(result)
    return np.real(result)

point = np.random.random(ansatz.num_parameters)
INDEX = 2

EPS = 0.2
# make identity vector with a 1 at index ``INDEX``, otherwise 0
e_i = np.identity(point.size)[:, INDEX]
print(e_i)

plus = point + EPS * e_i
minus = point - EPS * e_i

print(plus)
print(minus)

finite_difference = (
    evaluate_expectation(plus) - evaluate_expectation(minus)) / (2 * EPS)
print(finite_difference)

[0. 0. 1. 0.]
[0.70509724 0.3961315  1.13478115 0.34352652]
[0.70509724 0.3961315  0.73478115 0.34352652]
[0.5390625]
[0.68164062]
[-0.35644531]


### Nghich Quantum_info and primitives

In [4]:
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Clifford

X = SparsePauliOp("X")

qc = QuantumCircuit(1)
qc.h(0)
H = Clifford(qc).to_operator()

plus = QuantumCircuit(1)
plus.h(0)

estimator = Estimator()
values_plus = estimator.run([plus, plus], [X, H]).result().values

print(values_plus)

[1.         0.70710678]


## Calculate with Qiskit gradient

### opflow

In [11]:
from qiskit.opflow import Gradient

shifter = Gradient('fin_diff', analytic=False, epsilon=EPS)
grad = shifter.convert(expectation, params=ansatz.parameters[INDEX])
print(grad)

value_dict = dict(zip(ansatz.parameters, point))
print(value_dict)
sampler.convert(grad, value_dict).eval().real

C:\Users\vutha\AppData\Local\Temp\ipykernel_33344\263876192.py:3: DeprecationWarning: The class ``qiskit.opflow.gradients.gradient.Gradient`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  shifter = Gradient('fin_diff', analytic=False, epsilon=EPS)


SummedOp([
  2.5 * ComposedOp([
    OperatorMeasurement(ZZ),
    CircuitStateFn(
         ┌──────────┐     ┌────────────────┐
    q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2] + 0.2) ├
         ├──────────┤┌─┴─┐└──┬──────────┬──┘
    q_1: ┤ Ry(θ[1]) ├┤ X ├───┤ Ry(θ[3]) ├───
         └──────────┘└───┘   └──────────┘   
    )
  ]),
  -2.5 * ComposedOp([
    OperatorMeasurement(ZZ),
    CircuitStateFn(
         ┌──────────┐     ┌────────────────┐
    q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2] - 0.2) ├
         ├──────────┤┌─┴─┐└──┬──────────┬──┘
    q_1: ┤ Ry(θ[1]) ├┤ X ├───┤ Ry(θ[3]) ├───
         └──────────┘└───┘   └──────────┘   
    )
  ])
])
{ParameterVectorElement(θ[0]): 0.5456158771636148, ParameterVectorElement(θ[1]): 0.5306634836965907, ParameterVectorElement(θ[2]): 0.8753439889577473, ParameterVectorElement(θ[3]): 0.8250821839579529}


0.04760742187499978

### Quantum_info and primitives

In [23]:
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient

grad = ParamShiftEstimatorGradient(estimator)

print(ansatz)
print(hamiltonian)
print(point)

result = grad.run(ansatz, hamiltonian,[point]).result()
# print(result)

gradients = result.gradients
print(gradients)

     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├
     ├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├
     └──────────┘└───┘└──────────┘
ZZ
[0.54561588 0.53066348 0.87534399 0.82508218]
EstimatorGradientResult(gradients=[array([ 0.60573443, -0.56703284,  0.03897645, -0.32369708])], metadata=[{'parameters': ParameterView([ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3])])}], options=Options())
[array([ 0.60573443, -0.56703284,  0.03897645, -0.32369708])]
